# A Python Data Parsing and Visualization for VR Ball Catching Experiment

In [1]:
from __future__ import division

import pandas as pd
import numpy as np
from scipy import signal as sig

#import cv2
import os
import scipy.io as sio
import matplotlib

%matplotlib notebook


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


## Specify the subject folder list or the subject you want to run the analysis

In [2]:
fileTimeList = ['2016-4-19-14-4', '2016-4-22-11-57', '2016-4-27-13-28', '2016-4-28-10-57', '2016-4-29-11-56',
 '2016-5-3-12-52', '2016-5-4-13-3', '2016-5-5-13-7', '2016-5-6-11-2', '2016-5-6-13-4']

#fileTime = '2016-1-29-16-43'
# I picked the last subject here
fileTime = fileTimeList[-1]

## Modify the filePath according to where the datase is saved

In [3]:
expCfgName = "gd_pilot.cfg"
sysCfgName = "PERFORMVR.cfg"

filePath = "../ballcatchinganalysis/Data/" + fileTime + "/"
fileName = "exp_data-" + fileTime


## Let's read the data frames

In [4]:


sessionDict = pd.read_pickle(filePath + fileName + '.pickle')

rawDataFrame = sessionDict['raw']
processedDataFrame = sessionDict['processed']
calibDataFrame = sessionDict['calibration']
s1TrialInfo = sessionDict['trialInfo']

## Let's see what is inside the raw data frame

In [5]:
rawDataFrame[0:5]

frameNumber      IOD      IPD ballFinalPos              \
                                                  X        Y  Z   
frameNum                                                          
0              10797  64.8047  65.2305  -0.00862943  1.17583  0   
1              10798   64.806  65.2294  -0.00862943  1.17583  0   
2              10799  64.8073  65.2222  -0.00862943  1.17583  0   
3              10800  64.8073  65.2222  -0.00862943  1.17583  0   
4              10801   64.809  65.2311  -0.00862943  1.17583  0   

         ballInitialPos             ballInitialVel      ...         viewMat  \
                      X       Y   Z              X      ...              14   
frameNum                                                ...                   
0              -7.22369  1.8598  20         3.7974      ...       -0.109888   
1              -7.22369  1.8598  20         3.7974      ...       -0.112631   
2              -7.22369  1.8598  20         3.7974      ...       -0.115559   
3              -7.22369  1.8598  20         3.7974      ...       -0.118333   
4              -7.22369  1.8598  20         3.7974      ...       -0.121742   

              viewPos                       viewQuat                       \
         15         X        Y         Z           X         Y          Z   
frameNum                                                                    
0         1 -0.975283  1.58199 -0.109888 -0.00228223 -0.124441 -0.0207849   
1         1 -0.973299  1.58234 -0.112631  -0.0039463 -0.125008 -0.0206622   
2         1 -0.971469  1.58249 -0.115559 -0.00549355 -0.125491  -0.020463   
3         1 -0.969394  1.58294 -0.118333 -0.00698145 -0.125847 -0.0202717   
4         1 -0.967201  1.58364 -0.121742 -0.00841127 -0.126347 -0.0202151   

                        SubjectID  
                 W                 
frameNum                           
0         0.992007  2016-5-6-13-4  
1         0.991933  2016-5-6-13-4  
2         0.991868  2016-5-6-13-4  
3         0.991818  2016-5-6-13-4  
4         0.991744  2016-5-6-13-4  

[5 rows x 238 columns]

In [10]:
list(rawDataFrame.columns)

[('frameNumber', ''),
 ('IOD', ''),
 ('IPD', ''),
 ('ballFinalPos', 'X'),
 ('ballFinalPos', 'Y'),
 ('ballFinalPos', 'Z'),
 ('ballInitialPos', 'X'),
 ('ballInitialPos', 'Y'),
 ('ballInitialPos', 'Z'),
 ('ballInitialVel', 'X'),
 ('ballInitialVel', 'Y'),
 ('ballInitialVel', 'Z'),
 ('ballPos', 'X'),
 ('ballPos', 'Y'),
 ('ballPos', 'Z'),
 ('ballTTC', ''),
 ('ballVel', 'X'),
 ('ballVel', 'Y'),
 ('ballVel', 'Z'),
 ('blankDur', ''),
 ('blockNumber', ''),
 ('calibrationCounter', ''),
 ('calibrationPos', 'X'),
 ('calibrationPos', 'Y'),
 ('calibrationPos', 'Z'),
 ('cycEyeBasePoint', 'X'),
 ('cycEyeBasePoint', 'Y'),
 ('cycEyeBasePoint', 'Z'),
 ('cycEyeInHead', 'X'),
 ('cycEyeInHead', 'Y'),
 ('cycEyeInHead', 'Z'),
 ('cycEyeNodeInWorld', 'X'),
 ('cycEyeNodeInWorld', 'Y'),
 ('cycEyeNodeInWorld', 'Z'),
 ('cycEyeOnScreen', 'X'),
 ('cycEyeOnScreen', 'Y'),
 ('cycGazeNodeInWorld', 'X'),
 ('cycGazeNodeInWorld', 'Y'),
 ('cycGazeNodeInWorld', 'Z'),
 ('cycInverseMat', '0'),
 ('cycInverseMat', '1'),
 ('cycInve

### Here are the important columns that will be needed for reconstruction/visualization of the experiment

### Please note that all these values are in  <font color='red'>world coordinate system</font> 

In [24]:
rawDataFrame[['frameNumber', 'frameTime', 'ballPos', 'isBallVisibleQ', 'paddlePos','paddleQuat', 'viewPos', 'viewQuat']][0:5]

frameNumber frameTime  ballPos                   isBallVisibleQ  \
                                      X        Y        Z                  
frameNum                                                                   
0              10797    794.65 -7.17317  1.97791    19.86           True   
1              10798   794.663 -7.12262  2.09435  19.7198           True   
2              10799   794.676 -7.07201   2.2092  19.5796           True   
3              10800    794.69 -7.02143  2.32223  19.4394           True   
4              10801   794.703 -6.97078   2.4337  19.2989           True   

         paddlePos                    paddleQuat                      \
                 X        Y         Z          X         Y         Z   
frameNum                                                               
0         -0.68321  1.41418  0.212399   0.866933 -0.240231   0.36797   
1        -0.681674   1.4138  0.210206    0.86576 -0.241284  0.369471   
2        -0.680789  1.41331  0.208561   0.864943 -0.242207  0.370724   
3        -0.681506  1.41267  0.205969   0.863956 -0.243383  0.372415   
4        -0.679871   1.4121  0.204122   0.862789 -0.245152  0.374788   

                     viewPos                       viewQuat            \
                 W         X        Y         Z           X         Y   
frameNum                                                                
0        -0.235191 -0.975283  1.58199 -0.109888 -0.00228223 -0.124441   
1         -0.23608 -0.973299  1.58234 -0.112631  -0.0039463 -0.125008   
2        -0.236164 -0.971469  1.58249 -0.115559 -0.00549355 -0.125491   
3        -0.235907 -0.969394  1.58294 -0.118333 -0.00698145 -0.125847   
4        -0.234585 -0.967201  1.58364 -0.121742 -0.00841127 -0.126347   

                               
                  Z         W  
frameNum                       
0        -0.0207849  0.992007  
1        -0.0206622  0.991933  
2         -0.020463  0.991868  
3        -0.0202717  0.991818  
4        -0.0202151  0.991744

## Let's see what's inside processed data frame

In [22]:
processedDataFrame[0:5]

paddleFaceDir                     paddleUpDir                      \
              X         Y         Z           X         Y         Z   
0     -0.751011 -0.230994  0.618566    0.243441  0.773948  0.584586   
1     -0.753670 -0.230482  0.615516    0.243340  0.772096  0.587072   
2     -0.755711 -0.228952  0.613580    0.243888  0.771124  0.588121   
3     -0.758331 -0.226348  0.611311    0.244834  0.770226  0.588905   
4     -0.761743 -0.221036  0.609008    0.247189  0.769741  0.588555   

  paddlFaceLatDir                     paddleToBallVec      ...        \
                X         Y         Z               X      ...         
0       -0.613774  0.589615 -0.525010        -6.48996      ...         
1       -0.610547  0.592238 -0.525821        -6.44095      ...         
2       -0.607798  0.594094 -0.526909        -6.39122      ...         
3       -0.604144  0.596254 -0.528668        -6.33993      ...         
4       -0.598870  0.598867 -0.531707        -6.29091      ...         

  rotatedBallOnScreen         gazeError_HCS gazeError_WCS            \
                    Y       Z                           X         Y   
0         0.000518449  0.0725      2.792647     -0.283132  2.784054   
1         0.000701178  0.0725      2.659752     -0.345368  2.643217   
2         0.000903073  0.0725      2.510424     -0.400628  2.484409   
3          0.00111112  0.0725      2.348866     -0.437226  2.314067   
4          0.00132361  0.0725      2.189009     -0.497955  2.138006   

                gazeAngularError headVelocity ballVelocity      SubjectID  
              Z                                                            
0  1.834851e-10         2.792647      14.9217      24.6555  2016-5-6-13-4  
1  5.699773e-11         2.659752      13.6533      24.6749  2016-5-6-13-4  
2 -1.131400e-10         2.510424      12.8758      24.5888  2016-5-6-13-4  
3 -3.246579e-10         2.348866      12.8861      24.5065  2016-5-6-13-4  
4  3.028217e-10         2.189009      11.0966      24.6464  2016-5-6-13-4  

[5 rows x 68 columns]

In [23]:
list(processedDataFrame.columns)

[('paddleFaceDir', 'X'),
 ('paddleFaceDir', 'Y'),
 ('paddleFaceDir', 'Z'),
 ('paddleUpDir', 'X'),
 ('paddleUpDir', 'Y'),
 ('paddleUpDir', 'Z'),
 ('paddlFaceLatDir', 'X'),
 ('paddlFaceLatDir', 'Y'),
 ('paddlFaceLatDir', 'Z'),
 ('paddleToBallVec', 'X'),
 ('paddleToBallVec', 'Y'),
 ('paddleToBallVec', 'Z'),
 ('paddleToBallDir', 'X'),
 ('paddleToBallDir', 'Y'),
 ('paddleToBallDir', 'Z'),
 ('paddleToBallDirXZ', 'X'),
 ('paddleToBallDirXZ', 'Y'),
 ('paddleToBallDirXZ', 'Z'),
 ('paddleToBallLatDirXZ', 'X'),
 ('paddleToBallLatDirXZ', 'Y'),
 ('paddleToBallLatDirXZ', 'Z'),
 ('eventFlag', ''),
 ('frameTime', ''),
 ('trialNumber', ''),
 ('viewQuat', 'X'),
 ('viewQuat', 'Y'),
 ('viewQuat', 'Z'),
 ('viewQuat', 'W'),
 ('medFilt3_cycEyeOnScreen', 'X'),
 ('medFilt3_cycEyeOnScreen', 'Y'),
 ('medFilt5_cycEyeOnScreen', 'X'),
 ('medFilt5_cycEyeOnScreen', 'Y'),
 ('medFilt7_cycEyeOnScreen', 'X'),
 ('medFilt7_cycEyeOnScreen', 'Y'),
 ('avgFilt3_cycEyeOnScreen', 'X'),
 ('avgFilt3_cycEyeOnScreen', 'Y'),
 ('avgFi

### Here are the important columns that will be needed for reconstruction/visualization of the experiment

In [27]:
processedDataFrame[['gazePoint', 'ballOnScreen']][0:5]

gazePoint                     ballOnScreen                       
          X         Y         Z            X           Y          Z
0 -0.021715  0.004912  0.069238   -0.0215157  0.00137441  0.0693244
1 -0.021776  0.005151  0.069201   -0.0214932  0.00178959  0.0693186
2 -0.021828  0.005368  0.069168   -0.0214693  0.00220553  0.0693121
3 -0.021858  0.005568  0.069141    -0.021446  0.00261976  0.0693047
4 -0.021917  0.005761  0.069106   -0.0214214  0.00303309   0.069295

###  gazePoint : a 3D gaze vector w.r.t the <font color='red'>fixed</font>  head coordinate system (doesn't roate with head)
### ballOnScreen : a 3D eye-ball vector w.r.t the <font color='red'>fixed</font>  head coordinate system (doesn't roate with head)


In [29]:
processedDataFrame[['rotatedGazePoint', 'rotatedBallOnScreen']][0:5]

rotatedGazePoint                   rotatedBallOnScreen                     
                 X         Y       Z                   X            Y       Z
0        -0.004123  0.004044  0.0725         -0.00376431  0.000518449  0.0725
1        -0.004109  0.004048  0.0725         -0.00367245  0.000701178  0.0725
2        -0.004098  0.004049  0.0725         -0.00359137  0.000903073  0.0725
3        -0.004083  0.004041  0.0725         -0.00352942   0.00111112  0.0725
4        -0.004076  0.004030  0.0725         -0.00344567   0.00132361  0.0725

###  gazePoint : 3D gaze vector w.r.t the <font color='red'>rotating</font>  head coordinate <font color='red'>frame</font> (roates with the head)
### ballOnScreen : a 3D eye-ball vector w.r.t the <font color='red'>rotating</font>  head coordinate <font color='red'>frame</font> (roates with the head)
### Please note that the <font color='red'>Z values</font> are all 72.5 mm away from the eye which is the physical distance of the display to eye (on average)

### Also it is important to note that the magnitude of these vectors are not normalized so you simply need to use _scipy.linalg.norm_ to normalize the gaze vectors
